# **Code.txt** 

In [ ]:
#### Load the required packages
import os
import re

In [ ]:
#### Point it to the directory where your Test Data is present
input_dir = "/content/drive/MyDrive/sub3_ph22/"

In [ ]:
#### List all the folders present in the directory
list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]

In [ ]:
#### Load the Test Data
input_stanza_list = [] #Stores individual lines from Stanza_out.txt file 
input_stanza_len = [] #Stores the number of lines present in Stanza_out.txt file 
input_sent_num_list = [] #Stores individual lines from sentences.txt file
file_name_list = [] #Stores the name of individual files

for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    for files in os.listdir(input_dir + fls + '/' + i + "/"):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x,stanza_lines.splitlines()))) # filter empty strings and split into lines
        input_stanza_len.append(len(stanza_lines_list))
        input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(list(map(int, sentence_num_list)))
    count=count+1
         
    file_name_list.append(fls + '/' + str(i))
  print("completed",fls, count)

In [ ]:
print(len(input_sent_num_list))
print()
print(input_sent_num_list[1])
print()
print(len(file_name_list))
print()
print(file_name_list[1])
print()
print(len(input_stanza_list))
print()
print(input_stanza_list[1])

In [ ]:
#### Returns a list of urls present in a string using a regex expression. In case no url is present, it returns an empty list. 
def find_url(string): 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url] 

In [ ]:
#### The main function to identify where the Code information unit should be present and what should be its triples

for i in zip(input_sent_num_list, input_stanza_list, file_name_list):
  for sen_num in i[0]:

    # Get the contribution sentence corresponding to its sentence number in sentences.txt file
    j = i[1][(int)(sen_num)-1]  

    # Now we remove all the useless spaces which can be present in our url (eg. "https:// wikipedia .com" converted to "https://wikipedia.com")   
    # temp1 removes all leading and trailing spaces around "/"
    temp1 = re.sub(r'(?:(?<=\/) | (?=\/))','',j)
    # temp2 removes all leading and trailing spaces around ":"
    temp2 = re.sub(r'(?:(?<=\:) | (?=\:))','',temp1)
    # temp3 removes all leading and trailing spaces around "."
    temp3 = re.sub(r'(?:(?<=\.) | (?=\.))','',temp2)
    # temp4 removes all leading and trailing spaces around "-"
    temp4 = re.sub(r'(?:(?<=\-) | (?=\-))','',temp3)
    # Now temp4 is used as the final sentence in which urls have to be found

    # list_urls contains the list of urls found in that sentence
    list_urls = find_url(temp4)

    # If some url is found in that sentence, then proceed forward
    if (len(list_urls)!=0): 
      # Only those urls considered in Code Information unit whose corresponding sentence consists of words like "our", "code", "codes"
      if ("our" in j.lower().split(" ")) or ("code" in j.lower().split(" "))or ("codes" in j.lower().split(" ")):

        # Now we identify the starting and ending character number of the url in its respective sentence
        user_iterator = re.finditer('http', j.lower())

        # This list consists of the starting character number of the urls
        user_list = [user.start() for user in user_iterator]  
        
        for k in zip(user_list, list_urls):
          start = k[0]
          end = start 
          length_of_link = len(k[1])
          while (end<len(j))and(length_of_link!=0):
            # we ignore all the spaces occuring in between the url
            if (j[end]!=" "):
              length_of_link = length_of_link - 1
            # "end" stores the ending character number of the urls
            end = end + 1

          # Check the url in the original sentence using start and end character number   
          print(j[start:end])
          # Print the respective file name also
          print(i[2])

          # Write the extracted url along with its starting and ending character number, in the entities.txt file 
          output_file1 = open(input_dir + i[2] + "/entities.txt","a")
          output_file1.write(str(sen_num) + "\t" + str(start) + "\t" + str(end) + "\t" + j[start:end] + "\n")
          output_file1.close() 
          # Make and write the extracted url in the Code.txt file 
          output_file2 = open(input_dir + i[2] + "/triples/" + "code.txt","a")
          output_file2.write("(Contribution||Code||" + j[start:end] + ")" + "\n")
          output_file2.close()  

        print()      


# **Research_problem.txt**

In [ ]:
#### Load the required packages
import os
import re

In [ ]:
#### Point it to the directory where your Test Data is present
input_dir = "/content/drive/MyDrive/sub3_ph22/"

In [ ]:
#### List all the folders present in the directory
list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]

In [ ]:
#### Load Test Data
file_name_list = [] #Stores the name of all files 
input_entity_list = [] #Stores the list of all phrases from entities.txt

for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    for files in os.listdir(input_dir + fls + '/' + i + "/"):
      if files.endswith("entities.txt"):
        entities_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,(entities_file.read()).splitlines())) # filter empty strings and split into lines
        input_entity_list.append(entities_list)  
    count=count+1
    
    file_name_list.append(fls + '/' + str(i))
    
  print("completed",fls, count)

In [ ]:
#### Returns a list of urls present in a string using a regex expression. In case no url is present, it returns an empty list. 
def find_url(string): 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url] 

In [ ]:
for i in zip(file_name_list, input_entity_list):
  # print the file name
  print(i)
  # iterate over the phrases stored in entities.txt file
  for j in i[1]:
    # split them
    split_version = j.split('\t')

    # Open the empty Research-problem.txt file in that folder
    output_file = open(input_dir + i[0] + "/" + "triples/" + "research-problem.txt","a")
    
    # If the sentence number of that phrase is between "0 to 30", then only process further 
    if(0<(int)(split_version[0])<=30):

      # Now identify if that phrase is the "only phrase" extracted from that sentence 
      count_of_line = 0
      for kk in i[1]:
        if(kk.split('\t')[0]==split_version[0]):
          count_of_line += 1
          if(count_of_line == 2):
            break
            
      # "Count_of_line" stores the number of phrases extracted from that sentence.      
      if(count_of_line==1):      
        print(split_version)

        # It can happen that the single extracted phrase might be a url, so we check it here 
        list_urls = find_url(split_version[3])

        # Include it only if it is not a url 
        if (len(list_urls)==0): 
          output_file.write("(Contribution||has research problem||" + split_version[3] + ")" + "\n")
    output_file.close()  